In [1]:
import os, sys
import numpy as np
import torch
import espaloma as esp
import qcportal as ptl
from collections import Counter
from openff.toolkit.topology import Molecule
from openff.qcsubmit.results import BasicResultCollection
from simtk import unit
from simtk.unit import Quantity
from matplotlib import pyplot as plt

In [2]:
collection_type = "Dataset"
name = "RNA Single Point Dataset v1.0"

client = ptl.FractalClient()
collection = client.get_collection(collection_type, name)

#### check record

In [3]:
rna = client.get_collection(collection_type, name)
rna.list_records()

,driver,program,method,basis,keywords,name
0,gradient,psi4,wb97m-d3bj,def2-tzvppd,wb97m-d3bj/def2-tzvppd,WB97M-D3BJ/def2-tzvppd-wb97m-d3bj/def2-tzvppd
2,gradient,psi4,b3lyp,dzvp,default,B3LYP/dzvp-default
3,gradient,psi4,b3lyp-d3bj,dzvp,default,B3LYP-D3BJ/dzvp-default


In [4]:
#recs = collection.get_records(method='wb97m-d3bj', basis='def2-tzvppd', program='psi4', keywords='wb97m-d3bj/def2-tzvppd')
recs = collection.get_records(method='b3lyp-d3bj', basis='dzvp', program='psi4', keywords='default')
print("length of dataframe", len(recs), "\n", "No. of records in each: ",  len(recs[0]), len(recs[1]))

length of dataframe 2 
 No. of records in each:  4489 4489


#### inspect

In [5]:
# Sample dictionary of a record, and the properties that can be accessed through this
d3bj_correction_dataframe = recs[0]
# first record 
print(d3bj_correction_dataframe.iloc[0].record.dict())

{'id': '109521931', 'hash_index': None, 'procedure': 'single', 'program': 'dftd3', 'version': 1, 'protocols': {'wavefunction': <WavefunctionProtocolEnum.none: 'none'>}, 'extras': {'_qcfractal_tags': {'program': 'dftd3', 'keywords': None}, 'info': {'dashlevel': 'd3bj', 'dashparams': {'s6': 1.0, 's8': 1.9889, 'a1': 0.3981, 'a2': 4.4211}, 'fctldash': 'b3lyp-d3(bj)', 'dashparams_citation': ''}, 'local_keywords': {'dashlevel': 'd3bj', 'dashparams': {'s6': 1.0, 's8': 1.9889, 'a1': 0.3981, 'a2': 4.4211}, 'fctldash': 'b3lyp-d3(bj)', 'dashparams_citation': ''}, 'qcvars': {'CURRENT ENERGY': '-0.12241289', 'DISPERSION CORRECTION ENERGY': '-0.12241289', '2-BODY DISPERSION CORRECTION ENERGY': '-0.12241289', 'B3LYP-D3(BJ) DISPERSION CORRECTION ENERGY': '-0.12241289', 'CURRENT GRADIENT': array([[ 2.81302828e-04, -4.80197433e-04,  5.63011062e-04],
       [ 9.83013508e-05, -1.68913023e-04,  2.66803068e-04],
       [ 2.55541752e-04, -7.70356570e-04,  2.94138195e-04],
       [ 1.84664060e-05, -3.08778999

In [6]:
# Sample dictionary of a record, and the properties that can be accessed through this
b3lyp_dataframe = recs[1]
# first record 
print(b3lyp_dataframe.iloc[0].record.dict())

{'id': '109522131', 'hash_index': None, 'procedure': 'single', 'program': 'psi4', 'version': 1, 'protocols': {'wavefunction': <WavefunctionProtocolEnum.none: 'none'>}, 'extras': {'_qcfractal_tags': {'program': 'psi4', 'keywords': '34'}, 'qcvars': {'CURRENT DIPOLE X': 14.284629238711577, 'CURRENT DIPOLE Y': -6.273212669846057, 'CURRENT DIPOLE Z': 6.251830429771571, 'CURRENT ENERGY': -1782.5173009461919, 'CURRENT REFERENCE ENERGY': -1782.5173009461919, 'DFT FUNCTIONAL TOTAL ENERGY': -1782.5173009461928, 'DFT TOTAL ENERGY': -1782.5173009461928, 'DFT VV10 ENERGY': 0.0, 'DFT XC ENERGY': -189.75362775587155, 'GRID ELECTRONS ALPHA': 128.00005542252904, 'GRID ELECTRONS BETA': 128.00005542252904, 'GRID ELECTRONS TOTAL': 256.0001108450581, 'NUCLEAR REPULSION ENERGY': 3602.9164088388934, 'ONE-ELECTRON ENERGY': -9604.149126346882, 'PCM POLARIZATION ENERGY': 0.0, 'PE ENERGY': 0.0, 'SCF DIPOLE X': 14.284629238711577, 'SCF DIPOLE Y': -6.273212669846057, 'SCF DIPOLE Z': 6.251830429771571, 'SCF ITERATI

In [7]:
b3lyp_dataframe.record[0].dict()

{'id': '109522131',
 'hash_index': None,
 'procedure': 'single',
 'program': 'psi4',
 'version': 1,
 'protocols': {'wavefunction': <WavefunctionProtocolEnum.none: 'none'>},
 'extras': {'_qcfractal_tags': {'program': 'psi4', 'keywords': '34'},
  'qcvars': {'CURRENT DIPOLE X': 14.284629238711577,
   'CURRENT DIPOLE Y': -6.273212669846057,
   'CURRENT DIPOLE Z': 6.251830429771571,
   'CURRENT ENERGY': -1782.5173009461919,
   'CURRENT REFERENCE ENERGY': -1782.5173009461919,
   'DFT FUNCTIONAL TOTAL ENERGY': -1782.5173009461928,
   'DFT TOTAL ENERGY': -1782.5173009461928,
   'DFT VV10 ENERGY': 0.0,
   'DFT XC ENERGY': -189.75362775587155,
   'GRID ELECTRONS ALPHA': 128.00005542252904,
   'GRID ELECTRONS BETA': 128.00005542252904,
   'GRID ELECTRONS TOTAL': 256.0001108450581,
   'NUCLEAR REPULSION ENERGY': 3602.9164088388934,
   'ONE-ELECTRON ENERGY': -9604.149126346882,
   'PCM POLARIZATION ENERGY': 0.0,
   'PE ENERGY': 0.0,
   'SCF DIPOLE X': 14.284629238711577,
   'SCF DIPOLE Y': -6.27321

In [8]:
d3bj_correction_dataframe.record[0].dict()

{'id': '109521931',
 'hash_index': None,
 'procedure': 'single',
 'program': 'dftd3',
 'version': 1,
 'protocols': {'wavefunction': <WavefunctionProtocolEnum.none: 'none'>},
 'extras': {'_qcfractal_tags': {'program': 'dftd3', 'keywords': None},
  'info': {'dashlevel': 'd3bj',
   'dashparams': {'s6': 1.0, 's8': 1.9889, 'a1': 0.3981, 'a2': 4.4211},
   'fctldash': 'b3lyp-d3(bj)',
   'dashparams_citation': ''},
  'local_keywords': {'dashlevel': 'd3bj',
   'dashparams': {'s6': 1.0, 's8': 1.9889, 'a1': 0.3981, 'a2': 4.4211},
   'fctldash': 'b3lyp-d3(bj)',
   'dashparams_citation': ''},
  'qcvars': {'CURRENT ENERGY': '-0.12241289',
   'DISPERSION CORRECTION ENERGY': '-0.12241289',
   '2-BODY DISPERSION CORRECTION ENERGY': '-0.12241289',
   'B3LYP-D3(BJ) DISPERSION CORRECTION ENERGY': '-0.12241289',
   'CURRENT GRADIENT': array([[ 2.81302828e-04, -4.80197433e-04,  5.63011062e-04],
          [ 9.83013508e-05, -1.68913023e-04,  2.66803068e-04],
          [ 2.55541752e-04, -7.70356570e-04,  2.941

#### combine two energies

In [9]:
# Some calculations failed due to SCF convergence errors, so excluding those by checking the record status of b3lyp calculation
for i in range(len(recs[0])):
    if recs[1].iloc[i].record.status == 'COMPLETE':
        print("B3LYP + D3BJ energy of ", recs[1].iloc[i].record.id, recs[1].iloc[i].name, recs[1].iloc[i].record.properties.return_energy + recs[0].iloc[i].record.properties.return_energy)

B3LYP + D3BJ energy of  109522131 Nc1ccn([C@@H]2O[C@H](CO)[C@@H](O)[C@H]2O)c(=O)n1.Nc1ccn([C@@H]2O[C@H](CO)[C@@H](O)[C@H]2O)c(=O)n1-0 -1782.639713836192
B3LYP + D3BJ energy of  109522262 Nc1ccn([C@@H]2O[C@H](CO)[C@@H](O)[C@H]2O)c(=O)n1.Nc1ccn([C@@H]2O[C@H](CO)[C@@H](O)[C@H]2O)c(=O)n1-1 -1782.6302990214972
B3LYP + D3BJ energy of  109522115 Nc1ccn([C@@H]2O[C@H](CO)[C@@H](O)[C@H]2O)c(=O)n1.Nc1ccn([C@@H]2O[C@H](CO)[C@@H](O)[C@H]2O)c(=O)n1-2 -1782.641790205362
B3LYP + D3BJ energy of  109522209 Nc1ccn([C@@H]2O[C@H](CO)[C@@H](O)[C@H]2O)c(=O)n1.Nc1ccn([C@@H]2O[C@H](CO)[C@@H](O)[C@H]2O)c(=O)n1-3 -1782.62233649516
B3LYP + D3BJ energy of  109522302 Nc1ccn([C@@H]2O[C@H](CO)[C@@H](O)[C@H]2O)c(=O)n1.Nc1ccn([C@@H]2O[C@H](CO)[C@@H](O)[C@H]2O)c(=O)n1-4 -1782.648962953126
B3LYP + D3BJ energy of  109522167 Nc1ccn([C@@H]2O[C@H](CO)[C@@H](O)[C@H]2O)c(=O)n1.Nc1ccn([C@@H]2O[C@H](CO)[C@@H](O)[C@H]2O)c(=O)n1-5 -1782.6406730116719
B3LYP + D3BJ energy of  109522304 Nc1ccn([C@@H]2O[C@H](CO)[C@@H](O)[C@H]2O)c(=O)n

#### combine two gradient

In [10]:
recs[1].iloc[i].record.dict()

{'id': '109520662',
 'hash_index': None,
 'procedure': 'single',
 'program': 'psi4',
 'version': 1,
 'protocols': {'wavefunction': <WavefunctionProtocolEnum.none: 'none'>},
 'extras': {'_qcfractal_tags': {'program': 'psi4', 'keywords': '34'},
  'qcvars': {'CURRENT DIPOLE X': -295.52368171250646,
   'CURRENT DIPOLE Y': 409.1756484062543,
   'CURRENT DIPOLE Z': 3.6173396060994003,
   'CURRENT ENERGY': -3714.9768608244626,
   'CURRENT REFERENCE ENERGY': -3714.9768608244626,
   'DFT FUNCTIONAL TOTAL ENERGY': -3714.9768608244635,
   'DFT TOTAL ENERGY': -3714.9768608244635,
   'DFT VV10 ENERGY': 0.0,
   'DFT XC ENERGY': -352.46167409716026,
   'GRID ELECTRONS ALPHA': 222.00017279324246,
   'GRID ELECTRONS BETA': 222.00017279324246,
   'GRID ELECTRONS TOTAL': 444.0003455864849,
   'NUCLEAR REPULSION ENERGY': 8859.259098756938,
   'ONE-ELECTRON ENERGY': -22824.812958407165,
   'PCM POLARIZATION ENERGY': 0.0,
   'PE ENERGY': 0.0,
   'SCF DIPOLE X': -295.52368171250646,
   'SCF DIPOLE Y': 409.17

In [66]:
#assert recs[0].iloc[-1].record.properties.scf_total_energy == recs[0].iloc[-1].record.properties.return_energy
print(recs[0].iloc[-1].record.properties.scf_total_energy, recs[0].iloc[-1].record.properties.return_energy)

None -0.20735354


In [67]:
assert recs[1].iloc[-1].record.properties.scf_total_energy == recs[1].iloc[-1].record.properties.return_energy

In [68]:
return_result = recs[1].iloc[-1].record.return_result
current_grad = recs[1].iloc[-1].record.extras['qcvars']['CURRENT GRADIENT']
dft_total_grad = recs[1].iloc[-1].record.extras['qcvars']['DFT TOTAL GRADIENT']
scf_total_grad = recs[1].iloc[-1].record.extras['qcvars']['SCF TOTAL GRADIENT']

In [69]:
(return_result - current_grad).flatten()

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [70]:
(return_result - dft_total_grad).flatten()

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [71]:
(return_result - scf_total_grad).flatten()

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [72]:
# Some calculations failed due to SCF convergence errors, so excluding those by checking the record status of b3lyp calculation
print("B3LYP + D3BJ energy of ", recs[1].iloc[-1].record.id, recs[-1].iloc[i].name, recs[1].iloc[-1].record.return_result + recs[0].iloc[-1].record.return_result)

B3LYP + D3BJ energy of  109520662 O=c1ccn([C@@H]2O[C@H](COP(=O)([O-])O[C@@H]3[C@@H](COP(=O)([O-])O[C@@H]4[C@@H](CO)O[C@@H](n5ccc(=O)[nH]c5=O)[C@@H]4O)O[C@@H](n4ccc(=O)[nH]c4=O)[C@@H]3O)[C@@H](O)[C@H]2O)c(=O)[nH]1-9 [[ 1.76705103e-02  2.48895177e-03 -3.12531989e-03]
 [-3.43600956e-03 -3.01471929e-03  5.47336530e-03]
 [-7.05621045e-03  3.75466592e-03 -7.82407940e-03]
 [-5.01889192e-03  5.06548686e-03 -5.75004501e-04]
 [-2.10118405e-03 -5.27782328e-03 -4.45520959e-03]
 [-6.86186127e-03 -1.44607403e-03  2.13795178e-02]
 [-1.50102391e-03  1.33933152e-03 -1.34525889e-03]
 [ 1.53985901e-02 -1.19097370e-02 -4.21409668e-03]
 [-1.47447846e-03  8.76997055e-03 -2.09963537e-03]
 [-3.60138768e-03  8.10669943e-03 -1.37559923e-03]
 [ 3.02673852e-02 -5.21411874e-02 -8.35831894e-03]
 [ 5.18585097e-03  5.68717405e-03 -4.49071590e-03]
 [-1.15322596e-03 -3.98277075e-03  4.22703877e-03]
 [-1.86973132e-02  2.14449440e-03  1.67635100e-03]
 [ 9.37795136e-03 -3.58561292e-03 -1.34795703e-04]
 [-1.47781965e-03  1

In [73]:
recs[1].iloc[-1].record.return_result[:3]

array([[ 0.01728688,  0.00205116, -0.00244465],
       [-0.00354574, -0.00321086,  0.00577378],
       [-0.00779139,  0.00324594, -0.00748628]])

In [74]:
recs[0].iloc[-1].record.return_result[:3]

array([[ 0.00038363,  0.00043779, -0.00068067],
       [ 0.00010973,  0.00019614, -0.00030042],
       [ 0.00073518,  0.00050873, -0.0003378 ]])

#### convert mol to graph

In [75]:
i = -1
mol = client.query_molecules(recs[0].iloc[i].record.molecule)[0]
mol

NGLWidget()

In [76]:
offmol = Molecule.from_qcschema(mol, allow_undefined_stereo=True)   # convert to OpenFF Molecule object
offmol.compute_partial_charges_am1bcc()   # https://docs.openforcefield.org/projects/toolkit/en/0.9.2/api/generated/openff.toolkit.topology.Molecule.html

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 15, name: , idx: 28, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 27, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 29, aromatic: False, chiral: False
bond order: 2, chiral: False to atom atomic num: 8, name: , idx: 30, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 31, aromatic: False, chiral: False
Atom atomic num: 15, name: , idx: 58, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 57, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 59, aromatic: False, chiral: False
bond order: 2, chiral: False to atom atomic num: 8, name: , idx: 60, aromatic: False, chiral: False
bo

In [77]:
torch.tensor(offmol.partial_charges.value_in_unit(esp.units.CHARGE_UNIT))

tensor([-0.5622,  0.3857,  0.1125,  0.0082,  0.0082,  0.0966,  0.1489, -0.4368,
         0.1804,  0.0983, -0.5822,  0.1144,  0.0624, -0.5808,  0.4376,  0.2652,
         0.1247, -0.3635,  0.8083, -0.6184, -0.5942,  0.3410,  0.7215, -0.6601,
        -0.3701,  0.1633,  0.0853,  0.1740,  1.4328, -0.8183, -0.8183, -0.5609,
         0.1499,  0.0346,  0.0346,  0.1110,  0.1314, -0.4176,  0.1607,  0.0687,
        -0.5831,  0.1036,  0.0832, -0.5977,  0.4264,  0.2630,  0.1091, -0.3645,
         0.8058, -0.6565, -0.5970,  0.3346,  0.7178, -0.6647, -0.3537,  0.1806,
         0.0860,  0.2170,  1.4255, -0.8063, -0.8063, -0.5662,  0.1594,  0.0361,
         0.0361,  0.1313,  0.1017, -0.4254,  0.1320,  0.1092, -0.5925,  0.4181,
         0.0686,  0.0772, -0.5910,  0.4241,  0.2690,  0.1032, -0.3816,  0.8052,
        -0.6755, -0.5879,  0.3422,  0.7183, -0.6373, -0.3382,  0.1819,  0.0825,
         0.1998], dtype=torch.float64)

In [78]:
_energy1 = recs[0].iloc[i].record.properties.return_energy
_energy2 = recs[1].iloc[i].record.properties.return_energy

_grad1 = recs[0].iloc[i].record.return_result
_grad2 = recs[0].iloc[i].record.return_result

In [79]:
energy = _energy1 + _energy2
grad = _grad1 + _grad2

In [80]:
g = esp.Graph(offmol)
    
# energy is already hartree
e = torch.tensor(
    [
        Quantity(
            energy,
            esp.units.HARTREE_PER_PARTICLE,
        ).value_in_unit(esp.units.ENERGY_UNIT)
    ],
    dtype=torch.get_default_dtype(),
)[None, :]

xyz = torch.tensor(
    np.stack(
        [
            Quantity(
                mol.geometry,
                unit.bohr,
            ).value_in_unit(esp.units.DISTANCE_UNIT)
        ],
        axis=1,
    ),
    requires_grad=True,
    dtype=torch.get_default_dtype(),
)

u = torch.stack(
    [
        torch.tensor(
            Quantity(
                grad,
                esp.units.HARTREE_PER_PARTICLE / unit.bohr,
            ).value_in_unit(esp.units.FORCE_UNIT),
            dtype=torch.get_default_dtype(),
        )
    ],
    dim=1,
)

c = torch.tensor(offmol.partial_charges.value_in_unit(esp.units.CHARGE_UNIT), dtype=torch.get_default_dtype(),).unsqueeze(-1)

In [81]:
print(e.shape, xyz.shape, u.shape, c.shape)

torch.Size([1, 1]) torch.Size([89, 1, 3]) torch.Size([89, 1, 3]) torch.Size([89, 1])


#### save as pickle

In [82]:
# save as pickle object
import pickle
with open('offmol.pkl', 'wb') as pkl:
     pickle.dump(offmol, pkl, protocol=4)

In [83]:
with open('offmol.pkl', 'rb') as db:
    offmol_pkl = pickle.load(db)

In [84]:
offmol_pkl

NGLWidget()

In [85]:
offmol_pkl.partial_charges - offmol.partial_charges

Quantity(value=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.]), unit=elementary charge)